Libraries & Parameters

In [1]:
!pip install -q awswrangler

import awswrangler as wr
import pandas as pd
import boto3
import pytz
from datetime import datetime
from sagemaker import get_execution_role

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
# Get Sagemaker Role
role = get_execution_role()
print(role)

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210503T205912 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


arn:aws:iam::506926764659:role/service-role/AmazonSageMaker-ExecutionRole-20210503T205912


# 1.) Download Data

## S3 parameters

In [3]:
# # Flight data from 2007 and 2011 via Sagemaker Data Wrangler
# bucket = 'sagemaker-us-west-2-506926764659/export-flow-05-16-30-08-0c003aed/output/data-wrangler-flow-processing-05-16-30-08-0c003aed/b98f4f8c-ddaf-4ee1-99da-b0dd09f47a21/default'
# filename = 'part-00000-92fade68-00c4-41b3-9182-593084da2eae-c000.csv'
# path_to_file = 's3://{}/{}'.format(bucket, filename)

# Flight data from 2011_01 
bucket = 'from-public-data/carrier-perf/transformed/airOT2011'
filename = 'airOT201101.csv'
path_to_file = 's3://{}/{}'.format(bucket, filename)

# path_to_file

## Download data from S3

In [4]:
df = wr.s3.read_csv([path_to_file])
df

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 44
0,2011,1,1,6,2011-01-01,AA,N329AA,1,12478,JFK,...,334.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,1,2,7,2011-01-02,AA,N339AA,1,12478,JFK,...,362.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,1,3,1,2011-01-03,AA,N332AA,1,12478,JFK,...,359.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,1,4,2,2011-01-04,AA,N335AA,1,12478,JFK,...,341.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,1,5,3,2011-01-05,AA,N335AA,1,12478,JFK,...,340.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494395,2011,1,31,1,2011-01-31,YV,N989HA,2850,16218,YUM,...,52.0,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN
494396,2011,1,31,1,2011-01-31,YV,N434YV,2853,16218,YUM,...,48.0,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN
494397,2011,1,31,1,2011-01-31,YV,N989HA,2868,16218,YUM,...,45.0,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN
494398,2011,1,31,1,2011-01-31,YV,N77260,2870,16218,YUM,...,33.0,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN


# 2.) Features Engineering

In [6]:
# # Let's stick with (tried and trusted) SQL 
# # REF. https://www.kaggle.com/dataraj/sql-on-pandas-dataframe-using-pandasql
# !pip install pandasql

# import pandasql as ps

# query01 = "select * from df limit 12"

# # print(ps.sqldf(query01, locals()))

# sdf =  ps.sqldf(query01)
# sdf.head()

### Add new feature as target variable `is_late`

In [5]:
late_flight = []

for i in df['DEP_DELAY']:
    if i > 0 : late_flight.append('late')
    else: late_flight.append('not_late')
        
df['late_flight'] = late_flight
df

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 44,late_flight
0,2011,1,1,6,2011-01-01,AA,N329AA,1,12478,JFK,...,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,not_late
1,2011,1,2,7,2011-01-02,AA,N339AA,1,12478,JFK,...,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,not_late
2,2011,1,3,1,2011-01-03,AA,N332AA,1,12478,JFK,...,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,not_late
3,2011,1,4,2,2011-01-04,AA,N335AA,1,12478,JFK,...,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,not_late
4,2011,1,5,3,2011-01-05,AA,N335AA,1,12478,JFK,...,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,not_late
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494395,2011,1,31,1,2011-01-31,YV,N989HA,2850,16218,YUM,...,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN,not_late
494396,2011,1,31,1,2011-01-31,YV,N434YV,2853,16218,YUM,...,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN,not_late
494397,2011,1,31,1,2011-01-31,YV,N989HA,2868,16218,YUM,...,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN,not_late
494398,2011,1,31,1,2011-01-31,YV,N77260,2870,16218,YUM,...,1.0,160.0,1,NaN,NaN,NaN,NaN,NaN,NaN,not_late


### Select needed features

In [7]:
# REF. https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy

df_toTrain = df[[
    'YEAR',
    'MONTH',
    'DAY_OF_MONTH',
    'DAY_OF_WEEK',
    'UNIQUE_CARRIER',
    'ORIGIN_AIRPORT_ID',
    'ORIGIN',
    'DEST_AIRPORT_ID',
    'DEST',
    'DEP_TIME',
    'DEP_DELAY',
    'ACTUAL_ELAPSED_TIME',
    'ARR_TIME',
    'ARR_DELAY',
    'AIR_TIME',
    'DISTANCE',
    'CARRIER_DELAY',
    'WEATHER_DELAY',
    'NAS_DELAY',
    'SECURITY_DELAY',
    'LATE_AIRCRAFT_DELAY',
    'late_flight'
]].copy()

df_toTrain

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_TIME,...,ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,late_flight
0,2011,1,1,6,AA,12478,JFK,12892,LAX,906.0,...,1209.0,-26.0,334.0,2475.0,NaN,NaN,NaN,NaN,NaN,not_late
1,2011,1,2,7,AA,12478,JFK,12892,LAX,908.0,...,1236.0,1.0,362.0,2475.0,NaN,NaN,NaN,NaN,NaN,not_late
2,2011,1,3,1,AA,12478,JFK,12892,LAX,907.0,...,1239.0,4.0,359.0,2475.0,NaN,NaN,NaN,NaN,NaN,not_late
3,2011,1,4,2,AA,12478,JFK,12892,LAX,900.0,...,1217.0,-18.0,341.0,2475.0,NaN,NaN,NaN,NaN,NaN,not_late
4,2011,1,5,3,AA,12478,JFK,12892,LAX,904.0,...,1214.0,-21.0,340.0,2475.0,NaN,NaN,NaN,NaN,NaN,not_late
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494395,2011,1,31,1,YV,16218,YUM,14107,PHX,1910.0,...,2027.0,-4.0,52.0,160.0,NaN,NaN,NaN,NaN,NaN,not_late
494396,2011,1,31,1,YV,16218,YUM,14107,PHX,749.0,...,904.0,0.0,48.0,160.0,NaN,NaN,NaN,NaN,NaN,not_late
494397,2011,1,31,1,YV,16218,YUM,14107,PHX,1556.0,...,1704.0,-10.0,45.0,160.0,NaN,NaN,NaN,NaN,NaN,not_late
494398,2011,1,31,1,YV,16218,YUM,14107,PHX,1241.0,...,1333.0,-9.0,33.0,160.0,NaN,NaN,NaN,NaN,NaN,not_late


# 3.) Data Analysis

In [8]:
# Correlations

# 4.) Train Model

## Download PyCaret libraries

In [9]:
# Download PyCaret
!pip install pycaret

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached pycaret-2.3.1-py3-none-any.whl (261 kB)
  Using cached yellowbrick-1.3.post1-py3-none-any.whl (271 kB)
Processing /root/.cache/pip/wheels/01/e7/bb/347dc0e510803d7116a13d592b10cc68262da56a8eec4dd72f/umap_learn-0.5.1-py3-none-any.whl
  Using cached lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
  Using cached Boruta-0.3-py3-none-any.whl (56 kB)
  Using cached plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
  Using cached gensim-3.8.3-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
Processing /root/.cache/pip/wheels/

## Get the Data

In [10]:
data = df_toTrain.sample(frac=0.05, random_state=168)
data_unseen = df.drop(data.index)

data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (24720, 22)
Unseen Data For Predictions: (469680, 46)


## Setting Up Environment in PyCaret

In [11]:
from pycaret.classification import *
exp = setup(data = data, 
            target = 'late_flight',
            normalize = True,
            transformation = True,
            ignore_low_variance = True,
            remove_multicollinearity = True,
            multicollinearity_threshold = 0.95,
           )
# exp = setup(data = data, target = 'late_flight', session_id=123,
#             numeric_features = ['YR','MO','DAY_OF_MONTH','DAY_OF_WEEK']

,Description,Value
0,session_id,2779
1,Target,late_flight
2,Target Type,Binary
3,Label Encoded,"late: 0, not_late: 1"
4,Original Data,"(24720, 22)"
5,Missing Values,True
6,Numeric Features,14
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


## Comparing all models

In [12]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1130
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.7710
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,8.9860
lightgbm,Light Gradient Boosting Machine,0.9999,1.0000,1.0000,0.9999,1.0000,0.9999,0.9999,0.6720
rf,Random Forest Classifier,0.9994,1.0000,0.9998,0.9992,0.9995,0.9988,0.9988,3.1690
lr,Logistic Regression,0.8867,0.9727,0.9346,0.8859,0.9096,0.7583,0.7603,14.0230
et,Extra Trees Classifier,0.8753,0.9565,0.9489,0.8608,0.9027,0.7302,0.7370,8.5210
ridge,Ridge Classifier,0.8555,0.0000,0.9557,0.8322,0.8897,0.6832,0.6971,0.2030
lda,Linear Discriminant Analysis,0.8549,0.9401,0.9523,0.8335,0.8889,0.6823,0.6951,2.5220
svm,SVM - Linear Kernel,0.7417,0.0000,0.7954,0.8340,0.7470,0.4602,0.5146,2.7910


# 5.) Create Model(s)

## a. Decision Tree Classifier

In [13]:
dt = create_model('dt')

# trained model object is stored as `dt`
print(dt)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2779, splitter='best')


## b. ADA Boost Classifer

In [14]:
ada = create_model('ada')

# trained model object is stored as `ada`
print(ada)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=2779)
